In [18]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

In [19]:
class SemanticSearchTool:
    
    def __init__(self, codes_csv_path: str):
        embedding_file = 'embeddings.npy'
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.codes_df = pd.read_csv(codes_csv_path)
        print("Computing Vector Embedding for Code Descriptions")
        if os.path.exists(embedding_file):
            print("Loading embedding from the existing file")
            self.description_embeddings = np.load(embedding_file)
            print("Embeddings loaded Successfully")
        else:
            print("No Existing embeddings are here")
            self.description_embeddings = self.model.encode(self.codes_df["description"].tolist())
            np.save(embedding_file, self.description_embeddings)
            print(f"\nEmbeddings created and saved to {embedding_file}")
        
        print("Semantic Search Tools is ready....")

    def search(self, clinical_summary:str, top_k: int = 10) ->pd.DataFrame:
        if not clinical_summary:
            return pd.DataFrame()
        
        query_embedding = self.model.encode([clinical_summary])
        similarities = cosine_similarity(query_embedding, self.description_embeddings)
        top_k_indices = np.argsort(similarities[0])[-top_k:][::-1]
        return self.codes_df.iloc[top_k_indices]

In [21]:
if __name__ == '__main__':
    semantic_tool = SemanticSearchTool(codes_csv_path='icd10cm_codes_2026.csv')
    
    patient_summary = "Patient is 67-year old female who presented to the ermergency room with an acute onset of sharp, shooting lower abdominal pain, pronounced swelliing,nausea, vomiting, and a fever of 102°F. Physical examination revealed a firm, distended abdomen with tenderness to palpation, a positive Blumberg sign, and pain elicited by coughing, all highly suggestive of appendicitis. An ultrasound of the abdomen/pelvis confirmed the diagnosis of scute appendicitis. The patient underwent an uncomplicated laparoscopic appendectomy under general anesthesia. Intraoperatively, an inflamed appendix was found, but there was no evidence of localized or generalized peritonitis. The appendix was successfully resected"
    print(f"\nPerforming semantic search for: {patient_summary}")
    relevant_codes = semantic_tool.search(patient_summary)
    print(f'\nFound potentially relevant codes:')
    print(relevant_codes)

Computing Vector Embedding for Code Descriptions
Loading embedding from the existing file
Embeddings loaded Successfully
Semantic Search Tools is ready....

Performing semantic search for: Patient is 67-year old female who presented to the ermergency room with an acute onset of sharp, shooting lower abdominal pain, pronounced swelliing,nausea, vomiting, and a fever of 102°F. Physical examination revealed a firm, distended abdomen with tenderness to palpation, a positive Blumberg sign, and pain elicited by coughing, all highly suggestive of appendicitis. An ultrasound of the abdomen/pelvis confirmed the diagnosis of scute appendicitis. The patient underwent an uncomplicated laparoscopic appendectomy under general anesthesia. Intraoperatively, an inflamed appendix was found, but there was no evidence of localized or generalized peritonitis. The appendix was successfully resected

Found potentially relevant codes:
         code                                        description
11098     